In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve,least_squares,root
from scipy import optimize
from scipy.integrate import solve_ivp
import pandas as pd

In [2]:
A = np.array([[5/36, 2/9 - np.sqrt(15)/15, 5/36 - np.sqrt(15)/30],[5/36 + np.sqrt(15)/24, 2/9, 5/36 - np.sqrt(15)/24],[5/36 + np.sqrt(15)/30, 2/9 + np.sqrt(15)/15, 5/36]],dtype=np.float64)
c= np.array([1/2 - np.sqrt(15)/10, 1/2,1/2 + np.sqrt(15)/10],dtype=np.float64)
b= np.array([5/18, 4/9,5/18],dtype=np.float64)
b_tilda=np.array([-5/6,8/3,-5/6],dtype=np.float64)

In [3]:
A_e=np.array([[0,0,0,0],[1/3,0,0,0],[-1/3,1,0,0],[1,-1,1,0]],dtype=np.float64)
b_e=np.array([1/8,3/8,3/8,1/8],dtype=np.float64)
c_e=np.array([0,1/3,2/3,1],dtype=np.float64)

In [4]:
A_f=np.array([[0,0,0,0,0],[1/4,0,0,0,0],[3/32,9/32,0,0,0],[1932/2197,-7200/2197,7296/2197,0,0],[439/216,-8,3680/513,-845/4104,0],
              [-8/27,2,-3544/2565,1859/4104,-11/40]],
             dtype=np.float64)
b_f=np.array([16/135,0,6656/12825,28561/56430,-9/50,2/55],dtype=np.float64)
b_tilda_f=np.array([25/216,0,1408/2565,2197/4104,-1/5,0],dtype=np.float64)
c_f=np.array([0,1/4,3/8,12/13,1,1/2],dtype=np.float64)

In [5]:
A_f[5,4]

np.float64(-0.275)

In [6]:
def Thomas_Fermi(x,z):
    u,v=z
    return [v,np.power(u,1.5)/np.sqrt(x)]
def TF1(x,u,v):
     return v
def TF2(x,u,v):
    return np.power(u,1.5)/np.sqrt(x)

In [7]:
def Find_K(K,x_k,u_k,v_k,h):
    k1_u,k2_u,k3_u,k1_v,k2_v,k3_v=K
    eq1=k1_u-TF1(x_k+c[0]*h,u_k+h*(A[0,0]*k1_u+A[0,1]*k2_u+A[0,2]*k3_u),v_k+h*(A[0,0]*k1_v+A[0,1]*k2_v+A[0,2]*k3_v))
    eq2=k2_u-TF1(x_k+c[1]*h,u_k+h*(A[1,0]*k1_u+A[1,1]*k2_u+A[1,2]*k3_u),v_k+h*(A[1,0]*k1_v+A[1,1]*k2_v+A[1,2]*k3_v))
    eq3=k3_u-TF1(x_k+c[2]*h,u_k+h*(A[2,0]*k1_u+A[2,1]*k2_u+A[2,2]*k3_u),v_k+h*(A[2,0]*k1_v+A[2,1]*k2_v+A[2,2]*k3_v))
    eq4=k1_v-TF2(x_k+c[0]*h,u_k+h*(A[0,0]*k1_u+A[0,1]*k2_u+A[0,2]*k3_u),v_k+h*(A[0,0]*k1_v+A[0,1]*k2_v+A[0,2]*k3_v))
    eq5=k2_v-TF2(x_k+c[1]*h,u_k+h*(A[1,0]*k1_u+A[1,1]*k2_u+A[1,2]*k3_u),v_k+h*(A[1,0]*k1_v+A[1,1]*k2_v+A[1,2]*k3_v))
    eq6=k3_v-TF2(x_k+c[2]*h,u_k+h*(A[2,0]*k1_u+A[2,1]*k2_u+A[2,2]*k3_u),v_k+h*(A[2,0]*k1_v+A[2,1]*k2_v+A[2,2]*k3_v))
    return [eq1,eq2,eq3,eq4,eq5,eq6]

In [8]:
def Evaluate_K(h,K,u,v,point):
    sol=root(Find_K,K,args=(point,u,v,h))
    k1u,k2u,k3u,k1v,k2v,k3v=sol.x
    K=[k1u,k2u,k3u,k1v,k2v,k3v]
    return K

In [9]:
def Evaluate(K,u,v,h):
    k1u,k2u,k3u,k1v,k2v,k3v=K
    u_next=u+h*(b[0]*k1u+b[1]*k2u+b[2]*k3u)
    v_next=v+h*(b[0]*k1v+b[1]*k2v+b[2]*k3v) 
    u_tilda=u+h*(b_tilda[0]*k1u+b_tilda[1]*k2u+b_tilda[2]*k3u)
    v_tilda=v+h*(b_tilda[0]*k1v+b_tilda[1]*k2v+b_tilda[2]*k3v)
    return u_next,v_next,u_tilda,v_tilda

In [28]:
def Step_update(u_next,v_next,u_tilda,v_tilda,h,x_old,K,sol,dist,j,X,K_,epsilon_0,KK):
    step_error=np.sqrt(np.power(u_next-u_tilda,2)+np.power(v_next-v_tilda,2))
    x_curr=x_old
    n_iter=0
    n_iter1=0   
    if step_error<1e-6 and dist[j-1]>15:
        epsilon_0=np.power(10.0,-1)  
    while step_error>epsilon_0 and n_iter<35:
        
        h=beta*h*np.power((epsilon_0/step_error),0.2)
        h=min(h,1)
        x_try=x_old+h
        K=Evaluate_K(h,K,sol[j-1][0],sol[j-1][1],dist[j-1])
        u_next,v_next,u_tilda,v_tilda=Evaluate(K,sol[j-1][0],sol[j-1][1],h)
        step_error=np.sqrt(np.power(u_next-u_tilda,2)+np.power(v_next-v_tilda,2))
        n_iter+= 1
    if x_old+h>X:
        h=X-x_old
        K=Evaluate_K(h,K,sol[j-1][0],sol[j-1][1],dist[j-1])
        u_next,v_next,u_tilda,v_tilda=Evaluate(K,sol[j-1][0],sol[j-1][1],h)
        step_error=np.sqrt(np.power(u_next-u_tilda,2)+np.power(v_next-v_tilda,2))
        x_curr=X
    else:
        x_curr=x_old+h 
    if u_next>=0:
        h_new=beta_1*h*np.power((epsilon_0/step_error),1/2)
        dist.append(x_curr)
        sol.append([u_next, v_next])
        KK.append(K)
        h_new=min(h_new,5)
        return h_new
    else:
        while n_iter1 < 4:
            h = h / 10
            K = Evaluate_K(h, K_, sol[j-1][0], sol[j-1][1], dist[j-1])
            u_next, v_next, u_tilda, v_tilda = Evaluate(K, sol[j-1][0], sol[j-1][1], h)
            if u_next >= 0:
                break
            n_iter1 += 1
        else: 
            return 0
    
        x_curr = x_old + h
        h_new = beta * h * np.power((epsilon_0 / step_error), 1/2)
        KK.append(K)
        dist.append(x_curr)
        sol.append([u_next, v_next])
        return h_new

In [29]:
def explicit_K(h,u,v,point):
    k1u=TF1(point,u,v)
    k1v=TF2(point,u,v)
    k2u=TF1(point+c_e[1]*h,u+h*(A_e[1,0]*k1u),v+h*(A_e[1,0]*k1v))
    k2v=TF2(point+c_e[1]*h,u+h*(A_e[1,0]*k1u),v+h*(A_e[1,0]*k1v))
    k3u=TF1(point+c_e[2]*h,u+h*(A_e[2,0]*k1u+k2u*A_e[2,1]),v+h*(A_e[2,0]*k1v+k2v*A_e[2,1]))
    k3v=TF2(point+c_e[2]*h,u+h*(A_e[2,0]*k1u+k2u*A_e[2,1]),v+h*(A_e[2,0]*k1v+k2v*A_e[2,1]))
    k4u=TF1(point+c_e[3]*h,u+h*(A_e[3,0]*k1u+k2u*A_e[3,1]+A_e[3,2]*k3u),v+h*(A_e[3,0]*k1v+k2v*A_e[3,1]+k3v*A_e[3,2]))
    k4v=TF2(point+c_e[3]*h,u+h*(A_e[3,0]*k1u+k2u*A_e[3,1]+A_e[3,2]*k3u),v+h*(A_e[3,0]*k1v+k2v*A_e[3,1]+k3v*A_e[3,2]))
    K=[k1u,k2u,k3u,k4u,k1v,k2v,k3v,k4v]
    return K

In [30]:
def Fehlberg_K(h,u,v,point):
    k1u=TF1(point,u,v)
    k1v=TF2(point,u,v)
    k2u=TF1(point+c_f[1]*h,u+h*(A_f[1,0]*k1u),v+h*(A_f[1,0]*k1v))
    k2v=TF2(point+c_f[1]*h,u+h*(A_f[1,0]*k1u),v+h*(A_f[1,0]*k1v))
    k3u=TF1(point+c_f[2]*h,u+h*(A_f[2,0]*k1u+k2u*A_f[2,1]),v+h*(A_f[2,0]*k1v+k2v*A_f[2,1]))
    k3v=TF2(point+c_f[2]*h,u+h*(A_f[2,0]*k1u+k2u*A_f[2,1]),v+h*(A_f[2,0]*k1v+k2v*A_f[2,1]))
    k4u=TF1(point+c_f[3]*h,u+h*(A_f[3,0]*k1u+k2u*A_f[3,1]+A_f[3,2]*k3u),v+h*(A_f[3,0]*k1v+k2v*A_f[3,1]+k3v*A_f[3,2]))
    k4v=TF2(point+c_f[3]*h,u+h*(A_f[3,0]*k1u+k2u*A_f[3,1]+A_f[3,2]*k3u),v+h*(A_f[3,0]*k1v+k2v*A_f[3,1]+k3v*A_f[3,2]))
    k5u=TF1(point+c_f[4]*h,u+h*(A_f[4,0]*k1u+k2u*A_f[4,1]+A_f[4,2]*k3u+A_f[4,3]*k4u),v+h*(A_f[4,0]*k1v+k2v*A_f[4,1]+k3v*A_f[4,2]+k4v*A_f[4,3]))
    k5v=TF2(point+c_f[4]*h,u+h*(A_f[4,0]*k1u+k2u*A_f[4,1]+A_f[4,2]*k3u+A_f[4,3]*k4u),v+h*(A_f[4,0]*k1v+k2v*A_f[4,1]+k3v*A_f[4,2]+k4v*A_f[4,3]))
    k6u=TF1(point+c_f[5]*h,u+h*(A_f[5,0]*k1u+k2u*A_f[5,1]+A_f[5,2]*k3u+A_f[5,3]*k4u+A_f[5,4]*k5u),v+h*(A_f[5,0]*k1v+k2v*A_f[5,1]+k3v*A_f[5,2]+k4v*A_f[5,3]+k5v*A_f[5,4]))
    k6v=TF2(point+c_f[5]*h,u+h*(A_f[5,0]*k1u+k2u*A_f[5,1]+A_f[5,2]*k3u+A_f[5,3]*k4u+A_f[5,4]*k5u),v+h*(A_f[5,0]*k1v+k2v*A_f[5,1]+k3v*A_f[5,2]+k4v*A_f[5,3]+k5v*A_f[5,4]))
    K=[k1u,k2u,k3u,k4u,k5u,k6u,k1v,k2v,k3v,k4v,k5v,k6v]
    return K

In [31]:
def eval_explicit(h,u,v,point):
    K=Fehlberg_K(h,u,v,point)#point=distance[j-1]
    u_next=u+h*(b_f[0]*K[0]+b_f[1]*K[1]+b_f[2]*K[2]+b_f[3]*K[3]+b_f[4]*K[4]+b_f[5]*K[5])
    v_next=v+h*(b_f[0]*K[6]+b_f[1]*K[7]+b_f[2]*K[8]+b_f[3]*K[9]+b_f[4]*K[10]+b_f[5]*K[11])
    u_tilda=u+h*(b_tilda_f[0]*K[0]+b_tilda_f[1]*K[1]+b_tilda_f[2]*K[2]+b_tilda_f[3]*K[3]+b_tilda_f[4]*K[4]+b_tilda_f[5]*K[5])
    v_tilda=v+h*(b_tilda_f[0]*K[6]+b_tilda_f[1]*K[7]+b_tilda_f[2]*K[8]+b_tilda_f[3]*K[9]+b_tilda_f[4]*K[10]+b_tilda_f[5]*K[11])
    return u_next,v_next,u_tilda,v_tilda

In [32]:
def Step_explicit(u_next,v_next,u_tilda,v_tilda,error_threshold,h,x_old,sol,dist,j):
    step_error=np.sqrt(np.power(u_tilda-u_next,2)+np.power(v_tilda-v_next,2))
    iter_1=0
    x_curr=x_old
    iter_2=0
    while step_error>error_threshold and iter_1<30:
        h=beta*h*np.power(error_threshold/step_error,1/5)
        x_try=x_old+h
        u_next,v_next,u_tilda,_v_tilda=eval_explicit(h,sol[j-1][0],sol[j-1][1],dist[j-1])
        step_error=np.sqrt(np.power(u_tilda-u_next,2)+np.power(v_tilda-v_next,2))
        iter_1+=1
    h=min(h,0.08)
    if x_old+h>X:
        h=X-x_old
        u_next,v_next,u_tilda,_v_tilda=eval_explicit(h,sol[j-1][0],sol[j-1][1],dist[j-1])
        x_curr=X
    else: x_curr=x_old+h
    if u_next>=0:
        h_new=beta*h*np.power(error_threshold/step_error,1/4)
        sol.append([u_next,v_next])
        dist.append(x_curr)
        h_new=min(h,0.08)
        return h_new
    else:
        while iter_2<4:
            h=h/10
            u_next,v_next,u_tilda,_v_tilda=eval_explicit(h,sol[j-1][0],sol[j-1][1],dist[j-1])
            if u_next>0:
                break
            iter_2+=1
        else: return 0
        h_new=beta*h*np.power(error_threshold/step_error,1/4)
        x_curr=x_old+h
        sol.append([u_next,v_next])
        dist.append(x_curr)
        return h_new

In [33]:
def MY_Method(initial_step,x_0,X,u_0,v_0,epsilon_0):
    Solution=[]
    Dist=[]
    KK=[]
    switch=0
    distance=[x_0]
    sol=[[u_0,v_0]]
    h=initial_step
    K=np.zeros(6)
    K_=K
    h_curr=h
    j=1
    x_curr=x_0
    while distance[j-1]<X:
        x_curr=distance[j-1]
        
        K=Evaluate_K(h_curr,K_,sol[j-1][0],sol[j-1][1],distance[j-1])
        u_next,v_next,u_tilda,v_tilda=Evaluate(K,sol[j-1][0],sol[j-1][1],h_curr)
        h_curr=Step_update(u_next,v_next,u_tilda,v_tilda,h_curr,x_curr,K,sol,distance,j,X,K_,epsilon_0,KK)
        if h_curr!=0:
            j+=1
        else: break
        if np.allclose(KK[-1],0,1e-13):
            
            switch=1
            print(f"step={j}")
            #j-=1
            break
    if switch==1:
        print(f"Switching to RK_Exp, step={j},node={distance[-1]}")
        while distance[j-1]<X:
            x_curr=distance[j-1]
            
            u_next,v_next,u_tilda,v_tilda=eval_explicit(h,sol[j-1][0],sol[j-1][1],distance[j-1])
            
            h_curr=Step_explicit(u_next,v_next,u_tilda,v_tilda,1e-2,h_curr,x_curr,sol,distance,j)
        
            if h_curr!=0:
                j+=1
            else: break 
    sol=np.array(sol)
    distance=np.array(distance)
    return sol,distance,KK

In [34]:
x0=np.power(10.0,-235)
v_=-1.588071022611375312718684509
epsilon_0=np.power(10.0,-7)
beta=0.9
beta_1=0.9
X_list=[4,5,25,50]

In [35]:
method=['Radau','RK45','RK23','DOP853','LSODA']
df=pd.DataFrame(columns=['method','last node','value at the end','steps'])

In [36]:
for X in X_list:
    sol,d,KK=MY_Method(0.0001,0,X,1,v_,epsilon_0)
    df.loc[len(df)]=['My_method',d[-1],sol[-1,0],np.shape(d)[0]]

step=1105
Switching to RK_Exp, step=1105,node=45.32189219660929


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9616\4184207430.py:7: RuntimeWarning: invalid value encountered in power
  return np.power(u,1.5)/np.sqrt(x)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9616\1129645269.py:26: RuntimeWarning: divide by zero encountered in scalar divide
  h_new=beta_1*h*np.power((epsilon_0/step_error),1/2)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9616\824996737.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  h_new=beta*h*np.power(error_threshold/step_error,1/4)


In [37]:
for metoda in method:
    for X in X_list:
        solve3=solve_ivp(Thomas_Fermi,[x0,X],[1,v_],method=metoda,rtol=1e-13,atol=1e-13)
        df.loc[len(df)]=[metoda,solve3.t[-1],solve3.y[0,-1],np.shape(solve3.t)[0]]

In [38]:
df

,method,last node,value at the end,steps
0,My_method,4.0,0.108404,904
1,My_method,5.0,0.078807,931
2,My_method,25.0,0.003385,1079
3,My_method,50.0,0.000520,1164
4,Radau,4.0,0.108404,5464
5,Radau,5.0,0.078808,5627
6,Radau,25.0,0.003474,6619
7,Radau,50.0,0.000632,6897
8,RK45,4.0,0.108404,836
9,RK45,5.0,0.078808,865
